# SQL Agent with a Large Model on Amazon Bedrock

In our previous SQL Agents notebooks, we saw that our agents had some difficulty generating the correct SQL queries to answer some questions, especially when using small local models like Llama 3.2 3B. We will now look at creating a similar agent that generates SQL to answer a user's question, but use larger models that are available on [Amazon Bedrock](https://aws.amazon.com/bedrock/). For more details on signing up for Amazon Bedrock and its pricing, see [their documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-pricing.html). 

## Prerequisites

[Create an Amazon Bedrock API key](https://docs.aws.amazon.com/bedrock/latest/userguide/api-keys-generate.html), then add it to an `AWS_BEARER_TOKEN_BEDROCK` environment variable in your `.envrc` file.

We will use Pydantic AI to create our SQL agent. It should already be installed in your Python environment if you followed the installation instructions in the `README` file.


## Dataset: North Carolina General Statutes

We will use the same North Carolina General Statutes dataset that we've used in the previous notebooks.

In [1]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

import sys

if ".." not in sys.path:
    sys.path.insert(0, "..")

from utils import load_envrc

load_envrc("../../.envrc")

In [2]:
import pandas as pd

df = pd.read_csv("input/nc_general_statutes.csv.gz")
df.head(5)

,chapter_number,chapter_title,article_number,article,section_number,text,source_url
0,1,Civil Procedure,1,Definitions,1-1,§ 1-1. Remedies.\nRemedies in the courts of j...,https://www.ncleg.gov/EnactedLegislation/Statu...
1,1,Civil Procedure,1,Definitions,1-2,§ 1-2. Actions.\nAn action is an ordinary pro...,https://www.ncleg.gov/EnactedLegislation/Statu...
2,1,Civil Procedure,1,Definitions,1-3,§ 1-3. Special proceedings.\nEvery other reme...,https://www.ncleg.gov/EnactedLegislation/Statu...
3,1,Civil Procedure,1,Definitions,1-4,§ 1-4. Kinds of actions.\nActions are of two ...,https://www.ncleg.gov/EnactedLegislation/Statu...
4,1,Civil Procedure,1,Definitions,1-5,§ 1-5. Criminal action.\nA criminal action is...,https://www.ncleg.gov/EnactedLegislation/Statu...


In [3]:
# Now let's load the data into a PostgreSQL database.

import os
from sqlalchemy import create_engine

pg_engine = create_engine(os.getenv("DATABASE_URL"))

# Before inserting into DB, rename 'text' column to 'section_text' for clarity
df.rename(columns={"text": "section_text"}, inplace=True)

df.to_sql("nc_general_statutes", pg_engine, if_exists="replace", index=False)

-1

### Query the data

Let's think about different ways we can query the data. Some simple examples include:

- "How many chapters are there in total?"
- "What is the title of Chapter 15A?"
- "List all statutes in Chapter 15A."
- "What is the text of statute 15A-145?"

We can obviously ask more complex questions, but these simple ones will do for now. Let's try to answer these questions using SQL queries.

In [4]:
# "How many chapters are there in total?"

pd.read_sql("SELECT COUNT(DISTINCT chapter_number) FROM nc_general_statutes", pg_engine)

,count
0,395


In [5]:
# "What is the title of Chapter 15A?"

pd.read_sql(
    "SELECT DISTINCT chapter_title FROM nc_general_statutes WHERE chapter_number = '15A'", pg_engine
)

,chapter_title
0,Criminal Procedure Act


In [6]:
# "List all articles in Chapter 15A."

pd.read_sql(
    """
    SELECT DISTINCT 
        CAST(REGEXP_REPLACE(article_number, '[^0-9]', '', 'g') AS INTEGER) AS article_number_sort
        , article_number
        , article FROM nc_general_statutes
    WHERE chapter_number = '15A'
    """,
    pg_engine,
    dtype={"article_number_sort": "Int64"},
).head(20)

,article_number_sort,article_number,article
0,1,1,Definitions and General Provisions
1,2,2,Jurisdiction
2,3,3,Venue
3,4,4,Entry and Withdrawal of Attorney in Criminal Case
4,5,5,Expunction of Records
5,6,6,Certificate of Relief
6,7,7,Certificate of Relief
7,8,8,Electronic Recording of Interrogations
8,8,8A,SBI and State Crime Laboratory Access to View ...
9,9,9,Search and Seizure by Consent


## NC Statute SQL Agent

Now let's create a SQL agent that can answer questions about the North Carolina General Statutes. In addition to the questions above, ideally the agent can take certain questions further, for example: "Explain like I'm 5: Statute 15A-145".

### Agent instructions

We'll use LangChain's `SQLDatabase` tool to generate the database schema that we will use in the agent instructions, like we did in the SQL Agent notebooks. It's however possible to copy the code for the tool out of LangChain so that we don't have to install LangChain just for that.

In [7]:
# Add a comment to the section_text column in SQL: https://www.postgresql.org/docs/current/sql-comment.html
raw_conn = pg_engine.raw_connection()
with raw_conn.cursor() as conn:
    conn.execute(
        """
        COMMENT ON COLUMN nc_general_statutes.section_text IS 'The text of the statute section. Chapter and article text can be constructed by aggregating the section texts.';
        """
    )
    raw_conn.commit()

In [8]:
# Add comments for some columns, hopefully will make the agent use DISTINCT where necessary
raw_conn = pg_engine.raw_connection()
with raw_conn.cursor() as conn:
    conn.execute(
        """
        COMMENT ON COLUMN nc_general_statutes.chapter_number IS 'The chapter number. Each row in the table is a statute section, and the chapter number will be duplicated for all statute sections belonging to the same chapter.';
        COMMENT ON COLUMN nc_general_statutes.chapter_title IS 'The chapter title. Each row in the table is a statute section, and the chapter title will be duplicated for all statute sections belonging to the same chapter.';
        COMMENT ON COLUMN nc_general_statutes.article_number IS 'The article number. Each row in the table is a statute section, and the article number will be duplicated for all statute sections belonging to the same article.';
        COMMENT ON COLUMN nc_general_statutes.article IS 'The article title. Each row in the table is a statute section, and the article title will be duplicated for all statute sections belonging to the same article.';
        """
    )
    raw_conn.commit()

# Can use this to remove the comments if needed
raw_conn = pg_engine.raw_connection()
with raw_conn.cursor() as conn:
    conn.execute(
        """
        COMMENT ON COLUMN nc_general_statutes.chapter_number IS NULL;
        COMMENT ON COLUMN nc_general_statutes.chapter_title IS NULL;
        COMMENT ON COLUMN nc_general_statutes.article_number IS NULL;
        COMMENT ON COLUMN nc_general_statutes.article IS NULL;
        """
    )
    raw_conn.commit()

In [9]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase(engine=pg_engine)
SYSTEM_PROMPT = f"""
You are a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{db.get_table_info(get_col_comments=True)}

If the user asks about a 'statute', assume they mean 'section'.
"""

# Can comment this out to see the effect on the generated SQL
SYSTEM_PROMPT += """
Each row in the nc_general_statutes table is a statute section.
The values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.
The values in the article_number and article columns will be duplicated for all statute sections belonging to the same article. Use SELECT DISTINCT where necessary to get distinct values from these columns.
"""

print(SYSTEM_PROMPT)


You are a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:

CREATE TABLE nc_general_statutes (
	chapter_number TEXT, 
	chapter_title TEXT, 
	article_number TEXT, 
	article TEXT, 
	section_number TEXT, 
	section_text TEXT, 
	source_url TEXT
)

/*
Column Comments: {'section_text': 'The text of the statute section. Chapter and article text can be constructed by aggregating the section texts.'}
*/

/*
3 rows from nc_general_statutes table:
chapter_number	chapter_title	article_number	article	section_number	section_text	source_url
1	Civil Procedure	1	Definitions	1-1	§ 1-1. 

### Specifying a model

In order to specify the Amazon Bedrock model we want to use, we will provide a model ID when creating our agent. The list of models available on Bedrock, along with their model IDs, is available [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html). This page only lists models that are fully managed by Amazon, i.e. you don't have to deploy the model, set up endpoints, manage infra, etc. In addition to the models fully managed by Amazon (referred to as "Serverless" in the Model Catalog page), there are models that are available in the [Marketplace](https://docs.aws.amazon.com/bedrock/latest/userguide/amazon-bedrock-marketplace.html), which you can find in the Catalog, subscribe to them, then deploy them using Amazon SageMaker. 

![Model Catalog](./model_catalog.png)

Some models will require you to use [cross-Region inference](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html), in which case you should provide the inference profile ID instead of the model ID when specifying the model. The list of inference profile IDs is available [here](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-support.html), but it's usually the model ID prefixed with something like `us.` or `eu.` or `global.`. A list of inference profiles is also available in the console, which additionally lists the profile ARN that you can use instead of the profile ID when specifying the model.

![Model Catalog](./inference_profiles.png)

In Pydantic AI, we will prefix the model ID or inference profile ID with `bedrock:`.

In [10]:
# OpenAI gpt-oss models
# model = "bedrock:openai.gpt-oss-20b-1:0"
# model = "bedrock:openai.gpt-oss-120b-1:0"

# Llama 3.2 90B using its inference profile ID
# model = "bedrock:us.meta.llama3-2-90b-instruct-v1:0"
# Using inference profile ARN
# model = "bedrock:arn:aws:bedrock:us-east-1:779846792369:inference-profile/us.meta.llama3-2-90b-instruct-v1:0"

# Llama 3.3 70B
# model = "bedrock:us.meta.llama3-3-70b-instruct-v1:0"

# Llama 4 Maverick 17B
# model = "bedrock:us.meta.llama4-maverick-17b-instruct-v1:0"

# Cohere Command R+
model = "bedrock:cohere.command-r-plus-v1:0"

In [11]:
import os
from dataclasses import dataclass

import pandas as pd
import psycopg
from pydantic import BaseModel
from pydantic_ai import Agent, ModelRetry, RunContext
from sqlalchemy import Connection
from rich import print as rich_print


@dataclass
class Deps:
    conn: Connection
    db: SQLDatabase


class GenFailure(BaseModel):
    """Response if we could not generate a valid SELECT SQL query."""

    error_message: str


async def validate_sql_query(ctx: RunContext[Deps], sql_query: str) -> str:
    """Validate a SQL query on the database.

    Args:
        sql_query: The SQL query to be validated.
    """
    # gemini often adds extraneous backslashes to SQL
    sql_query = sql_query.replace("\\", "")
    if not sql_query.upper().startswith("SELECT"):
        raise ModelRetry("Please create a SELECT query")

    try:
        await ctx.deps.conn.execute(f"EXPLAIN {sql_query}")
    except psycopg.Error as e:
        raise ModelRetry(f"Invalid query: {e}") from e
    else:
        return sql_query


sql_gen_agent = Agent(
    model,
    instructions=SYSTEM_PROMPT,
    # Type ignore while we wait for PEP-0747, nonetheless unions will work fine everywhere else
    output_type=[validate_sql_query, GenFailure],  # type: ignore
    deps_type=Deps,
    retries=2,
)


@dataclass
class ExecDeps:
    conn: Connection
    db: SQLDatabase
    sql_query: str


class ExecFailure(BaseModel):
    """Response the user that the SQL query failed to execute."""

    error_message: str


sql_exec_agent = Agent(
    model,
    instructions="You are a helpful assistant. Use the available tools to run the user's SQL query and reply to their question.",
    # Type ignore while we wait for PEP-0747, nonetheless unions will work fine everywhere else
    output_type=[str, ExecFailure],  # type: ignore
    deps_type=ExecDeps,
    retries=2,
)


@sql_exec_agent.system_prompt
def exec_agent_system_prompt(ctx: RunContext):
    return f"""
    You are a PostgreSQL expert with read-only access to a database. Answer the user's question by running this SQL query exactly as it is:
    {ctx.deps.sql_query}
    
    This is the schema of the database:
    {ctx.deps.db.get_table_info()}
    """


@sql_exec_agent.tool
async def run_sql_query(ctx: RunContext[ExecDeps], sql_query: str) -> list:
    """Run a SQL query.

    Args:
        sql_query: The SQL query to be executed.
    """
    async with ctx.deps.conn.cursor() as acur:
        await acur.execute(sql_query)
        rows = await acur.fetchall()
        return rows


async def prompt(question):
    db = SQLDatabase.from_uri(os.getenv("DATABASE_URL"))

    async with await psycopg.AsyncConnection.connect(
        os.getenv("DATABASE_URL").replace("+psycopg", "")
    ) as aconn:
        deps = Deps(conn=aconn, db=db)
        result = await sql_gen_agent.run(question, deps=deps)

        print(f"SQL generation result: {result.output}")
        if isinstance(result.output, GenFailure):
            return ExecFailure(error_message=result.output.error_message)

        print("-" * 50)
        deps = ExecDeps(conn=aconn, db=db, sql_query=result.output)
        result = await sql_exec_agent.run(question, deps=deps)
        print(result.output)

In [12]:
# await prompt("What is the title of Chapter 15A?")

I found that iterating over an agent's graph, using the iterator returned by `Agent.iter()`, helps me see what the agent is actually doing. 

In [13]:
async def prompt_iter(question):
    db = SQLDatabase.from_uri(os.getenv("DATABASE_URL"), sample_rows_in_table_info=0)

    async with await psycopg.AsyncConnection.connect(
        os.getenv("DATABASE_URL").replace("+psycopg", "")
    ) as aconn:
        print("GEN AGENT RUN")
        print("-" * 20)
        deps = Deps(conn=aconn, db=db)
        async with sql_gen_agent.iter(question, deps=deps) as gen_agent_run:
            async for node in gen_agent_run:
                rich_print(node)
        print("-" * 50)
        print(f"SQL generation result: {gen_agent_run.result.output}")

        if isinstance(gen_agent_run.result.output, GenFailure):
            print(f"Error generating the SQL query: {gen_agent_run.result.output.error_message}")
            return

        print("EXEC AGENT RUN")
        print("-" * 20)
        deps = ExecDeps(conn=aconn, db=db, sql_query=gen_agent_run.result.output)
        async with sql_exec_agent.iter(question, deps=deps) as exec_agent_run:
            async for node in exec_agent_run:
                rich_print(node)
        print("=" * 50)
        print(exec_agent_run.result.output)

In [14]:
await prompt_iter("What is the title of Chapter 15A?")

GEN AGENT RUN
--------------------


UserPromptNode(user_prompt='What is the title of Chapter 15A?', instructions="You are a PostgreSQL expert with 
read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query 
to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).\n\nPay 
attention to use only the column names that you can see in the schema description. Be careful to not query for 
columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly use the following 
tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, \n\tarticle_number 
TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url TEXT\n)\n\n/*\nColumn 
Comments: {'section_text': 'The text of the statute section. Chapter and article text can be constructed by 
aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns.", instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the title of Chapter 15A?', 
timestamp=datetime.datetime(2025, 11, 7, 13, 26, 49, 710737, tzinfo=datetime.timezone.utc))], instructions="You are
a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct 
PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, 
DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema description. Be 
careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly 
use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, 
\n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url 
TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and article text can be 
constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will write a SQL query to find the title of 
Chapter 15A.'), ToolCallPart(tool_name='final_result_validate_sql_query', args={'sql_query': "SELECT DISTINCT 
chapter_title FROM nc_general_statutes WHERE chapter_number = '15A';"}, 
tool_call_id='tooluse_pmuUCl4WTIu1ubwWxPrLWg')], usage=RequestUsage(input_tokens=644, output_tokens=55), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 26, 53, 582128, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='ffc0a4c7-f222-4a86-a01b-8479e6607b96', finish_reason='tool_call'))

End(
    data=FinalResult(output="SELECT DISTINCT chapter_title FROM nc_general_statutes WHERE chapter_number = '15A';",
tool_name='final_result_validate_sql_query', tool_call_id='tooluse_pmuUCl4WTIu1ubwWxPrLWg')
)

...

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will run the SQL query provided by the user 
to find the answer to their question.'), ToolCallPart(tool_name='run_sql_query', args={'sql_query': "SELECT 
DISTINCT chapter_title FROM nc_general_statutes WHERE chapter_number = '15A';"}, 
tool_call_id='tooluse_ozeXjjSMSZmXUYOP9DdHqg')], usage=RequestUsage(input_tokens=190, output_tokens=52), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 26, 57, 432495, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='4ee00e2e-f374-488f-ad31-381c8f4768f1', finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='run_sql_query', content=[('Criminal 
Procedure Act',)], tool_call_id='tooluse_ozeXjjSMSZmXUYOP9DdHqg', timestamp=datetime.datetime(2025, 11, 7, 13, 26, 
57, 449890, tzinfo=datetime.timezone.utc))], instructions="You are a helpful assistant. Use the available tools to 
run the user's SQL query and reply to their question."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The title of Chapter 15A is the Criminal 
Procedure Act.')], usage=RequestUsage(input_tokens=226, output_tokens=14), model_name='cohere.command-r-plus-v1:0',
timestamp=datetime.datetime(2025, 11, 7, 13, 26, 59, 828068, tzinfo=datetime.timezone.utc), 
provider_name='bedrock', provider_details={'finish_reason': 'end_turn'}, 
provider_response_id='59bba33a-79d0-4b9c-8a61-801d634dcbe5', finish_reason='stop'))

End(data=FinalResult(output='The title of Chapter 15A is the Criminal Procedure Act.'))

The title of Chapter 15A is the Criminal Procedure Act.


In [15]:
await prompt_iter("List all articles in Chapter 15A including article numbers.")

GEN AGENT RUN
--------------------


UserPromptNode(user_prompt='List all articles in Chapter 15A including article numbers.', instructions="You are a 
PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct 
PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, 
DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema description. Be 
careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly 
use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, 
\n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url 
TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and article text can be 
constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns.", instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='List all articles in Chapter 15A including 
article numbers.', timestamp=datetime.datetime(2025, 11, 7, 13, 26, 59, 949941, tzinfo=datetime.timezone.utc))], 
instructions="You are a PostgreSQL expert with read-only access to the database. Given an input question, create a 
syntactically correct PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries
(INSERT, UPDATE, DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema 
description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which
table.\n\nOnly use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, 
\n\tchapter_title TEXT, \n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, 
\n\tsource_url TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and 
article text can be constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will write a SQL query to find all articles 
in Chapter 15A including article numbers.'), ToolCallPart(tool_name='final_result_validate_sql_query', 
args={'sql_query': "SELECT DISTINCT article_number, article FROM nc_general_statutes WHERE chapter_number = 
'15A';"}, tool_call_id='tooluse_8BBISxVnQ3i5KNoBCcjmrw')], usage=RequestUsage(input_tokens=646, output_tokens=60), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 27, 3, 205553, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='228813e2-fb20-44f7-ae15-72a8cceb518c', finish_reason='tool_call'))

End(
    data=FinalResult(output="SELECT DISTINCT article_number, article FROM nc_general_statutes WHERE chapter_number 
= '15A';", tool_name='final_result_validate_sql_query', tool_call_id='tooluse_8BBISxVnQ3i5KNoBCcjmrw')
)

...

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will run the SQL query provided by the user 
to answer their request.'), ToolCallPart(tool_name='run_sql_query', args={'sql_query': "SELECT DISTINCT 
article_number, article FROM nc_general_statutes WHERE chapter_number = '15A';"}, 
tool_call_id='tooluse_a9CUQpRyTPW8OVlSSYlnCA')], usage=RequestUsage(input_tokens=194, output_tokens=51), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 27, 6, 177376, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='ddf82a45-4d4c-4cb7-98b8-abdc9530284b', finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='run_sql_query', content=[('1', 'Definitions 
and General Provisions'), ('10', 'Other Searches and Seizures'), ('100', 'Capital Punishment'), ('101', 'North 
Carolina Racial Justice Act'), ('11', 'Search Warrants'), ('12', 'Pen Registers; Trap and Trace Devices'), ('13', 
'DNA Database and Databank'), ('14', 'Nontestimonial Identification'), ('14A', 'Eyewitness Identification Reform 
Act'), ('15', 'Urgent Necessity'), ('16', 'Electronic Surveillance'), ('16A', 'Discontinuation of 
Telecommunications Services'), ('16B', 'Use of Unmanned Aircraft Systems'), ('17', 'Criminal Process'), ('18', 
'Identification Documents'), ('19', 'Identification Documents'), ('2', 'Jurisdiction'), ('20', 'Arrest'), ('21', 
'Arrest'), ('22', 'Arrest'), ('23', 'Police Processing and Duties upon Arrest'), ('24', 'Initial Appearance'), 
('25', 'Commitment'), ('26', 'Bail'), ('27', 'Bail'), ('28', 'Bail'), ('29', 'First Appearance Before District 
Court Judge'), ('3', 'Venue'), ('30', 'Probable-Cause Hearing'), ('31', 'The Grand Jury and Its Proceedings'), 
('32', 'Indictment and Related Instruments'), ('33', 'Indictment and Related Instruments'), ('34', 'Indictment and 
Related Instruments'), ('35', 'Speedy Trial'), ('36', 'Special Criminal Process for Attendance of Defendants'), 
('37', 'Uniform Criminal Extradition Act'), ('38', 'Interstate Agreement on Detainers'), ('39', 'Other Special 
Process for Attendance of Defendants'), ('4', 'Entry and Withdrawal of Attorney in Criminal Case'), ('40', 'Other 
Special Process for Attendance of Defendants'), ('41', 'Other Special Process for Attendance of Defendants'), 
('42', 'Attendance of Witnesses Generally'), ('43', 'Uniform Act to Secure Attendance of Witnesses from Without a 
State in Criminal Proceedings'), ('44', 'Securing Attendance of Prisoners as Witnesses'), ('45', 'Fair Treatment 
for Certain Victims and Witnesses'), ('46', "Crime Victims' Rights Act"), ('47', "Crime Victims' Rights Act"), 
('48', 'Discovery in the Superior Court'), ('49', 'Pleadings and Joinder'), ('5', 'Expunction of Records'), ('50', 
'Voluntary Dismissal'), ('51', 'Arraignment'), ('52', 'Motions Practice'), ('53', 'Motion to Suppress Evidence'), 
('54', 'Reliability of In-Custody Informant Statements'), ('55', 'Reliability of In-Custody Informant Statements'),
('56', 'Incapacity to Proceed'), ('57', 'Pleas'), ('58', 'Procedures Relating to Guilty Pleas in Superior Court'), 
('58A', 'Procedures Relating to Felony Guilty Pleas in District Court'), ('59', 'Maintenance of Order in the 
Courtroom'), ('6', 'Certificate of Relief'), ('60', 'Maintenance of Order in the Courtroom'), ('61', 'Granting of 
Immunity to Witnesses'), ('62', 'Mistrial'), ('63', 'Mistrial'), ('64', 'Mistrial'), ('65', 'In General'), ('66', 
'Procedure for Hearing and Disposition of Infractions'), ('7', 'Certificate of Relief'), ('71', 'Right to Trial by 
Jury'), ('72', 'Selecting and Impaneling the Jury'), ('73', 'Criminal Jury Trial in Superior Court'), ('74', 
'Criminal Jury Trial in Superior Court'), ('75', 'Criminal Jury Trial in Superior Court'), ('76', 'Criminal Jury 
Trial in Superior Court'), ('77', 'Criminal Jury Trial in Superior Court'), ('78', 'Order of Commitment to 
Imprisonment'), ('79', 'Order of Commitment to Imprisonment'), ('8', 'Electronic Recording of Interrogations'), 
('80', 'Defendants Found Not Guilty by Reason of Insanity'), ('81', 'General Sentencing Provisions'), ('81A', 
'Sentencing Persons Convicted of Felonies'), ('81B', 'Structured Sentencing of Persons Convicted of Crimes'), 
('81C', 'Restitution'), ('81D', 'Permanent No Contact Order Against Convicted Sex Offender'), ('82', 'Probation'), 
('83', 'Imprisonment'), ('84', 'Fines'), ('84A', 'Post-Release Supervision'), ('84B', 'Medical Release of 
Inmates'), ('85', 'Parole'), ('85A', 'Parole of Certain Convicted Felons'), ('85B', 'Review of Sentences of Life 
Imprisonment Without Parole'), ('86', 'Reports of D

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="Here is a list of all articles in Chapter 15A, 
including article numbers:\n\n| Article Number | Article |\n|---|---|\n| 1 | Definitions and General Provisions 
|\n| 10 | Other Searches and Seizures |\n| 100 | Capital Punishment |\n| 101 | North Carolina Racial Justice Act 
|\n| 11 | Search Warrants |\n| 12 | Pen Registers; Trap and Trace Devices |\n| 13 | DNA Database and Databank |\n| 
14 | Nontestimonial Identification |\n| 14A | Eyewitness Identification Reform Act |\n| 15 | Urgent Necessity |\n| 
16 | Electronic Surveillance |\n| 16A | Discontinuation of Telecommunications Services |\n| 16B | Use of Unmanned 
Aircraft Systems |\n| 17 | Criminal Process |\n| 18 | Identification Documents |\n| 19 | Identification Documents 
|\n| 2 | Jurisdiction |\n| 20 | Arrest |\n| 21 | Arrest |\n| 22 | Arrest |\n| 23 | Police Processing and Duties 
upon Arrest |\n| 24 | Initial Appearance |\n| 25 | Commitment |\n| 26 | Bail |\n| 27 | Bail |\n| 28 | Bail |\n| 29 
| First Appearance Before District Court Judge |\n| 3 | Venue |\n| 30 | Probable-Cause Hearing |\n| 31 | The Grand 
Jury and Its Proceedings |\n| 32 | Indictment and Related Instruments |\n| 33 | Indictment and Related Instruments 
|\n| 34 | Indictment and Related Instruments |\n| 35 | Speedy Trial |\n| 36 | Special Criminal Process for 
Attendance of Defendants |\n| 37 | Uniform Criminal Extradition Act |\n| 38 | Interstate Agreement on Detainers 
|\n| 39 | Other Special Process for Attendance of Defendants |\n| 4 | Entry and Withdrawal of Attorney in Criminal 
Case |\n| 40 | Other Special Process for Attendance of Defendants |\n| 41 | Other Special Process for Attendance of
Defendants |\n| 42 | Attendance of Witnesses Generally |\n| 43 | Uniform Act to Secure Attendance of Witnesses from
Without a State in Criminal Proceedings |\n| 44 | Securing Attendance of Prisoners as Witnesses |\n| 45 | Fair 
Treatment for Certain Victims and Witnesses |\n| 46 | Crime Victims' Rights Act |\n| 47 | Crime Victims' Rights Act
|\n| 48 | Discovery in the Superior Court |\n| 49 | Pleadings and Joinder |\n| 5 | Expunction of Records |\n| 50 | 
Voluntary Dismissal |\n| 51 | Arraignment |\n| 52 | Motions Practice |\n| 53 | Motion to Suppress Evidence |\n| 54 
| Reliability of In-Custody Informant Statements |\n| 55 | Reliability of In-Custody Informant Statements |\n| 56 |
Incapacity to Proceed |\n| 57 | Pleas |\n| 58 | Procedures Relating to Guilty Pleas in Superior Court |\n| 58A | 
Procedures Relating to Felony Guilty Pleas in District Court |\n| 59 | Maintenance of Order in the Courtroom |\n| 6
| Certificate of Relief |\n| 60 | Maintenance of Order in the Courtroom |\n| 61 | Granting of Immunity to Witnesses
|\n| 62 | Mistrial |\n| 63 | Mistrial |\n| 64 | Mistrial |\n| 65 | In General |\n| 66 | Procedure for Hearing and 
Disposition of Infractions |\n| 7 | Certificate of Relief |\n| 71 | Right to Trial by Jury |\n| 72 | Selecting and 
Impaneling the Jury |\n| 73 | Criminal Jury Trial in Superior Court |\n| 74 | Criminal Jury Trial in Superior Court
|\n| 75 | Criminal Jury Trial in Superior Court |\n| 76 | Criminal Jury Trial in Superior Court |\n| 77 | Criminal 
Jury Trial in Superior Court |\n| 78 | Order of Commitment to Imprisonment |\n| 79 | <co: 0>Order of Commitment")],
usage=RequestUsage(input_tokens=1300, output_tokens=968), model_name='cohere.command-r-plus-v1:0', 
timestamp=datetime.datetime(2025, 11, 7, 13, 28, 54, 822596, tzinfo=datetime.timezone.utc), 
provider_name='bedrock', provider_details={'finish_reason': 'max_tokens'}, 
provider_response_id='665b4b1f-e134-47d3-9614-c1a573b2429a', finish_reason='length'))

End(
    data=FinalResult(output="Here is a list of all articles in Chapter 15A, including article numbers:\n\n| Article
Number | Article |\n|---|---|\n| 1 | Definitions and General Provisions |\n| 10 | Other Searches and Seizures |\n| 
100 | Capital Punishment |\n| 101 | North Carolina Racial Justice Act |\n| 11 | Search Warrants |\n| 12 | Pen 
Registers; Trap and Trace Devices |\n| 13 | DNA Database and Databank |\n| 14 | Nontestimonial Identification |\n| 
14A | Eyewitness Identification Reform Act |\n| 15 | Urgent Necessity |\n| 16 | Electronic Surveillance |\n| 16A | 
Discontinuation of Telecommunications Services |\n| 16B | Use of Unmanned Aircraft Systems |\n| 17 | Criminal 
Process |\n| 18 | Identification Documents |\n| 19 | Identification Documents |\n| 2 | Jurisdiction |\n| 20 | 
Arrest |\n| 21 | Arrest |\n| 22 | Arrest |\n| 23 | Police Processing and Duties upon Arrest |\n| 24 | Initial 
Appearance |\n| 25 | Commitment |\n| 26 | Bail |\n| 27 | Bail |\n| 28 | Bail |\n| 29 | First Appearance Before 
District Court Judge |\n| 3 | Venue |\n| 30 | Probable-Cause Hearing |\n| 31 | The Grand Jury and Its Proceedings 
|\n| 32 | Indictment and Related Instruments |\n| 33 | Indictment and Related Instruments |\n| 34 | Indictment and 
Related Instruments |\n| 35 | Speedy Trial |\n| 36 | Special Criminal Process for Attendance of Defendants |\n| 37 
| Uniform Criminal Extradition Act |\n| 38 | Interstate Agreement on Detainers |\n| 39 | Other Special Process for 
Attendance of Defendants |\n| 4 | Entry and Withdrawal of Attorney in Criminal Case |\n| 40 | Other Special Process
for Attendance of Defendants |\n| 41 | Other Special Process for Attendance of Defendants |\n| 42 | Attendance of 
Witnesses Generally |\n| 43 | Uniform Act to Secure Attendance of Witnesses from Without a State in Criminal 
Proceedings |\n| 44 | Securing Attendance of Prisoners as Witnesses |\n| 45 | Fair Treatment for Certain Victims 
and Witnesses |\n| 46 | Crime Victims' Rights Act |\n| 47 | Crime Victims' Rights Act |\n| 48 | Discovery in the 
Superior Court |\n| 49 | Pleadings and Joinder |\n| 5 | Expunction of Records |\n| 50 | Voluntary Dismissal |\n| 51
| Arraignment |\n| 52 | Motions Practice |\n| 53 | Motion to Suppress Evidence |\n| 54 | Reliability of In-Custody 
Informant Statements |\n| 55 | Reliability of In-Custody Informant Statements |\n| 56 | Incapacity to Proceed |\n| 
57 | Pleas |\n| 58 | Procedures Relating to Guilty Pleas in Superior Court |\n| 58A | Procedures Relating to Felony
Guilty Pleas in District Court |\n| 59 | Maintenance of Order in the Courtroom |\n| 6 | Certificate of Relief |\n| 
60 | Maintenance of Order in the Courtroom |\n| 61 | Granting of Immunity to Witnesses |\n| 62 | Mistrial |\n| 63 |
Mistrial |\n| 64 | Mistrial |\n| 65 | In General |\n| 66 | Procedure for Hearing and Disposition of Infractions 
|\n| 7 | Certificate of Relief |\n| 71 | Right to Trial by Jury |\n| 72 | Selecting and Impaneling the Jury |\n| 73
| Criminal Jury Trial in Superior Court |\n| 74 | Criminal Jury Trial in Superior Court |\n| 75 | Criminal Jury 
Trial in Superior Court |\n| 76 | Criminal Jury Trial in Superior Court |\n| 77 | Criminal Jury Trial in Superior 
Court |\n| 78 | Order of Commitment to Imprisonment |\n| 79 | <co: 0>Order of Commitment")
)

Here is a list of all articles in Chapter 15A, including article numbers:

| Article Number | Article |
|---|---|
| 1 | Definitions and General Provisions |
| 10 | Other Searches and Seizures |
| 100 | Capital Punishment |
| 101 | North Carolina Racial Justice Act |
| 11 | Search Warrants |
| 12 | Pen Registers; Trap and Trace Devices |
| 13 | DNA Database and Databank |
| 14 | Nontestimonial Identification |
| 14A | Eyewitness Identification Reform Act |
| 15 | Urgent Necessity |
| 16 | Electronic Surveillance |
| 16A | Discontinuation of Telecommunications Services |
| 16B | Use of Unmanned Aircraft Systems |
| 17 | Criminal Process |
| 18 | Identification Documents |
| 19 | Identification Documents |
| 2 | Jurisdiction |
| 20 | Arrest |
| 21 | Arrest |
| 22 | Arrest |
| 23 | Police Processing and Duties upon Arrest |
| 24 | Initial Appearance |
| 25 | Commitment |
| 26 | Bail |
| 27 | Bail |
| 28 | Bail |
| 29 | First Appearance Before District Court Judge |
| 3 | Venue |
| 30 | Pro

In [16]:
await prompt_iter("How many chapters are there in total?")

GEN AGENT RUN
--------------------


UserPromptNode(user_prompt='How many chapters are there in total?', instructions="You are a PostgreSQL expert with 
read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query 
to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).\n\nPay 
attention to use only the column names that you can see in the schema description. Be careful to not query for 
columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly use the following 
tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, \n\tarticle_number 
TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url TEXT\n)\n\n/*\nColumn 
Comments: {'section_text': 'The text of the statute section. Chapter and article text can be constructed by 
aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns.", instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='How many chapters are there in total?', 
timestamp=datetime.datetime(2025, 11, 7, 13, 28, 54, 935991, tzinfo=datetime.timezone.utc))], instructions="You are
a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct 
PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, 
DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema description. Be 
careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly 
use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, 
\n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url 
TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and article text can be 
constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will write a SQL query to count the number of
distinct chapter numbers in the nc_general_statutes table.'), 
ToolCallPart(tool_name='final_result_validate_sql_query', args={'sql_query': 'SELECT COUNT(DISTINCT chapter_number)
FROM nc_general_statutes;'}, tool_call_id='tooluse_2ULElf98SKagNkyvFBwBKA')], usage=RequestUsage(input_tokens=641, 
output_tokens=56), model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 28, 58, 
5057, tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='1aae2e83-b43f-44d2-8169-ca15b126dc1e', finish_reason='tool_call'))

End(
    data=FinalResult(output='SELECT COUNT(DISTINCT chapter_number) FROM nc_general_statutes;', 
tool_name='final_result_validate_sql_query', tool_call_id='tooluse_2ULElf98SKagNkyvFBwBKA')
)

...

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will run the SQL query provided by the user 
to find the answer to their question.'), ToolCallPart(tool_name='run_sql_query', args={'sql_query': 'SELECT 
COUNT(DISTINCT chapter_number) FROM nc_general_statutes;'}, tool_call_id='tooluse_iCjMba8MTv28qt3_mk3B6w')], 
usage=RequestUsage(input_tokens=181, output_tokens=46), model_name='cohere.command-r-plus-v1:0', 
timestamp=datetime.datetime(2025, 11, 7, 13, 29, 0, 831189, tzinfo=datetime.timezone.utc), provider_name='bedrock',
provider_details={'finish_reason': 'tool_use'}, provider_response_id='be33581c-3b2b-49f1-b02a-f94a6cf2d3a0', 
finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='run_sql_query', content=[(395,)], 
tool_call_id='tooluse_iCjMba8MTv28qt3_mk3B6w', timestamp=datetime.datetime(2025, 11, 7, 13, 29, 0, 858529, 
tzinfo=datetime.timezone.utc))], instructions="You are a helpful assistant. Use the available tools to run the 
user's SQL query and reply to their question."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='There are 395 chapters in total.')], 
usage=RequestUsage(input_tokens=216, output_tokens=10), model_name='cohere.command-r-plus-v1:0', 
timestamp=datetime.datetime(2025, 11, 7, 13, 29, 3, 13646, tzinfo=datetime.timezone.utc), provider_name='bedrock', 
provider_details={'finish_reason': 'end_turn'}, provider_response_id='043b7fe2-321b-4d54-ac58-52b30a1a53bf', 
finish_reason='stop'))

End(data=FinalResult(output='There are 395 chapters in total.'))

There are 395 chapters in total.


In [17]:
await prompt_iter("What is the text of statute 15A-145?")

GEN AGENT RUN
--------------------


UserPromptNode(user_prompt='What is the text of statute 15A-145?', instructions="You are a PostgreSQL expert with 
read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query 
to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).\n\nPay 
attention to use only the column names that you can see in the schema description. Be careful to not query for 
columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly use the following 
tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, \n\tarticle_number 
TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url TEXT\n)\n\n/*\nColumn 
Comments: {'section_text': 'The text of the statute section. Chapter and article text can be constructed by 
aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns.", instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the text of statute 15A-145?', 
timestamp=datetime.datetime(2025, 11, 7, 13, 29, 3, 107668, tzinfo=datetime.timezone.utc))], instructions="You are 
a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct 
PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, 
DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema description. Be 
careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly 
use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, 
\n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url 
TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and article text can be 
constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will write a SQL query to find the text of 
statute 15A-145.'), ToolCallPart(tool_name='final_result_validate_sql_query', args={'sql_query': "SELECT 
section_text FROM nc_general_statutes WHERE section_number = '15A-145';"}, 
tool_call_id='tooluse_09NohY0eR9KNQ8__6uNCfw')], usage=RequestUsage(input_tokens=648, output_tokens=61), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 29, 6, 393940, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='13aa7c6f-68af-4f03-a0fe-8bf4f4eac44e', finish_reason='tool_call'))

End(
    data=FinalResult(output="SELECT section_text FROM nc_general_statutes WHERE section_number = '15A-145';", 
tool_name='final_result_validate_sql_query', tool_call_id='tooluse_09NohY0eR9KNQ8__6uNCfw')
)

...

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will run the SQL query provided by the user 
and write an answer based on the results.'), ToolCallPart(tool_name='run_sql_query', args={'sql_query': "SELECT 
section_text FROM nc_general_statutes WHERE section_number = '15A-145';"}, 
tool_call_id='tooluse_bhlBC-0dQdy_pG7Lv8XS5A')], usage=RequestUsage(input_tokens=196, output_tokens=55), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 29, 9, 466036, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='ad4a9e7f-155b-4dd2-aa02-fb1ac5a4c625', finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='run_sql_query', content=[("§ 15A-145. 
\xa0Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; 
expunction of certain other misdemeanors.\n(a)\tWhenever any person who has not previously been convicted of any 
felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State 
or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the 
offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a 
misdemeanor possession of alcohol pursuant to G.S.\xa018B-302(b)(1), and the offense was committed before the 
person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for 
expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years 
after the date of the conviction, or (ii) the completion of any period of probation, whichever occurs later, and 
the petition shall contain, but not be limited to, the following:\n(1)\tAn affidavit by the petitioner that he has 
been of good behavior for the two-year period since the date of conviction of the misdemeanor in question and has 
not been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United 
States or the laws of this State or any other state.\n(2)\tVerified affidavits of two persons who are not related 
to the petitioner or to each other by blood or marriage, that they know the character and reputation of the 
petitioner in the community in which he lives and that his character and reputation are good.\n(3)\tA statement 
that the petition is a motion in the cause in the case wherein the petitioner was convicted.\n(4)\tRepealed by 
Session Laws 2010-174, s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or 
after that date.\n(4a)\tAn application on a form approved by the Administrative Office of the Courts requesting and
authorizing a name-based State and national criminal record check by the State Bureau of Investigation using any 
information required by the Administrative Office of the Courts to identify the individual and a search of the 
confidential record of expunctions maintained by the Administrative Office of the Courts. The application shall be 
filed with the clerk of superior court. The clerk of superior court shall forward the application to the State 
Bureau of Investigation and to the Administrative Office of the Courts, which shall conduct the searches and report
their findings to the court.\n(5)\tAn affidavit by the petitioner that no restitution orders or civil judgments 
representing amounts ordered for restitution entered against him are outstanding.\nThe petition shall be served 
upon the district attorney of the court wherein the case was tried resulting in conviction. The district attorney 
shall have 10 days thereafter in which to file any objection thereto and shall be duly notified as to the date of 
the hearing of the petition.\nThe judge to whom the petition is presented is authorized to call upon a probation 
officer for any additional investigation or verification of the petitioner's conduct during the two-year period 
that he deems desirable.\n(a1)\tNothing in this section shall be interpreted to allow the expunction of any offense
involving impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration pursuant to 
Article 27A of Chapter 14 of the General Statutes, whether or not the person is currently required to 
register.\n(b)\tIf the court, after hearing, finds that the petitioner had remained of good behavior and been free 
of conviction of any felony or misdemeanor, other than a traffic violation, for two years from the date of 
conviction of the misdemeanor in question,

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="Here is the text of statute 15A-145:\n\n> § 
15A-145. Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; 
expunction of certain other misdemeanors.\n> (a)\xa0Whenever any person who has not previously been convicted of 
any felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this 
State or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and 
the offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of 
a misdemeanor possession of alcohol pursuant to G.S.\xa018B-302(b)(1), and the offense was committed before the 
person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for 
expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years 
after the date of the conviction, or (ii) the completion of any period of probation, whichever occurs later, and 
the petition shall contain, but not be limited to, the following:\n> (1)\xa0An affidavit by the petitioner that he 
has been of good behavior for the two-year period since the date of conviction of the misdemeanor in question and 
has not been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United 
States or the laws of this State or any other state.\n> (2)\xa0Verified affidavits of two persons who are not 
related to the petitioner or to each other by blood or marriage, that they know the character and reputation of the
petitioner in the community in which he lives and that his character and reputation are good.\n> (3)\xa0A statement
that the petition is a motion in the cause in the case wherein the petitioner was convicted.\n> (4)\xa0Repealed by 
Session Laws 2010-174, s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or 
after that date.\n> (4a)\xa0An application on a form approved by the Administrative Office of the Courts requesting
and authorizing a name-based State and national criminal record check by the State Bureau of Investigation using 
any information required by the Administrative Office of the Courts to identify the individual and a search of the 
confidential record of expunctions maintained by the Administrative Office of the Courts. The application shall be 
filed with the clerk of superior court. The clerk of superior court shall forward the application to the State 
Bureau of Investigation and to the Administrative Office of the Courts, which shall conduct the searches and report
their findings to the court.\n> (5)\xa0An affidavit by the petitioner that no restitution orders or civil judgments
representing amounts ordered for restitution entered against him are outstanding.\n> The petition shall be served 
upon the district attorney of the court wherein the case was tried resulting in conviction. The district attorney 
shall have 10 days thereafter in which to file any objection thereto and shall be duly notified as to the date of 
the hearing of the petition.\n> The judge to whom the petition is presented is authorized to call upon a probation 
officer for any additional investigation or verification of the petitioner's conduct during the two-year period 
that he deems desirable.\n> (a1)\xa0Nothing in this section shall be interpreted to allow the expunction of any 
offense involving impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration 
pursuant to Article 27A of Chapter 14 of the General Statutes, whether or not the person is currently required to 
register.\n> (b)\xa0If the court, after hearing, finds that the petitioner had remained of good behavior and been 
free of conviction of any felony or misdemeanor, other than a traffic violation, for two years from the date 

End(
    data=FinalResult(output="Here is the text of statute 15A-145:\n\n> § 15A-145. Expunction of records for first 
offenders under the age of 18 at the time of conviction of misdemeanor; expunction of certain other 
misdemeanors.\n> (a)\xa0Whenever any person who has not previously been convicted of any felony, or misdemeanor 
other than a traffic violation, under the laws of the United States, the laws of this State or any other state, (i)
pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the offense was committed before
the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of 
alcohol pursuant to G.S.\xa018B-302(b)(1), and the offense was committed before the person attained the age of 21 
years, he may file a petition in the court of the county where he was convicted for expunction of the misdemeanor 
from his criminal record. The petition cannot be filed earlier than: (i) two years after the date of the 
conviction, or (ii) the completion of any period of probation, whichever occurs later, and the petition shall 
contain, but not be limited to, the following:\n> (1)\xa0An affidavit by the petitioner that he has been of good 
behavior for the two-year period since the date of conviction of the misdemeanor in question and has not been 
convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United States or the 
laws of this State or any other state.\n> (2)\xa0Verified affidavits of two persons who are not related to the 
petitioner or to each other by blood or marriage, that they know the character and reputation of the petitioner in 
the community in which he lives and that his character and reputation are good.\n> (3)\xa0A statement that the 
petition is a motion in the cause in the case wherein the petitioner was convicted.\n> (4)\xa0Repealed by Session 
Laws 2010-174, s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or after that 
date.\n> (4a)\xa0An application on a form approved by the Administrative Office of the Courts requesting and 
authorizing a name-based State and national criminal record check by the State Bureau of Investigation using any 
information required by the Administrative Office of the Courts to identify the individual and a search of the 
confidential record of expunctions maintained by the Administrative Office of the Courts. The application shall be 
filed with the clerk of superior court. The clerk of superior court shall forward the application to the State 
Bureau of Investigation and to the Administrative Office of the Courts, which shall conduct the searches and report
their findings to the court.\n> (5)\xa0An affidavit by the petitioner that no restitution orders or civil judgments
representing amounts ordered for restitution entered against him are outstanding.\n> The petition shall be served 
upon the district attorney of the court wherein the case was tried resulting in conviction. The district attorney 
shall have 10 days thereafter in which to file any objection thereto and shall be duly notified as to the date of 
the hearing of the petition.\n> The judge to whom the petition is presented is authorized to call upon a probation 
officer for any additional investigation or verification of the petitioner's conduct during the two-year period 
that he deems desirable.\n> (a1)\xa0Nothing in this section shall be interpreted to allow the expunction of any 
offense involving impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration 
pursuant to Article 27A of Chapter 14 of the General Statutes, whether or not the person is currently required to 
register.\n> (b)\xa0If the court, after hearing, finds that the petitioner had remained of good behavior and been 
free of conviction of any felony or misdemeanor, other than a traffic violation, for two years from the date of 
conviction of the misdemeanor

Here is the text of statute 15A-145:

> § 15A-145. Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; expunction of certain other misdemeanors.
> (a) Whenever any person who has not previously been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to G.S. 18B-302(b)(1), and the offense was committed before the person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years after the date of the conviction, or (ii) the completion of

In [18]:
await prompt_iter("Explain like I'm 5: Statute 15A-145")

GEN AGENT RUN
--------------------


UserPromptNode(user_prompt="Explain like I'm 5: Statute 15A-145", instructions="You are a PostgreSQL expert with 
read-only access to the database. Given an input question, create a syntactically correct PostgreSQL SELECT query 
to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, DELETE, DROP etc.).\n\nPay 
attention to use only the column names that you can see in the schema description. Be careful to not query for 
columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly use the following 
tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, \n\tarticle_number 
TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url TEXT\n)\n\n/*\nColumn 
Comments: {'section_text': 'The text of the statute section. Chapter and article text can be constructed by 
aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns.", instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content="Explain like I'm 5: Statute 15A-145", 
timestamp=datetime.datetime(2025, 11, 7, 13, 30, 58, 902155, tzinfo=datetime.timezone.utc))], instructions="You are
a PostgreSQL expert with read-only access to the database. Given an input question, create a syntactically correct 
PostgreSQL SELECT query to run to help find the answer. You should never make any DML queries (INSERT, UPDATE, 
DELETE, DROP etc.).\n\nPay attention to use only the column names that you can see in the schema description. Be 
careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nOnly 
use the following tables:\n\nCREATE TABLE nc_general_statutes (\n\tchapter_number TEXT, \n\tchapter_title TEXT, 
\n\tarticle_number TEXT, \n\tarticle TEXT, \n\tsection_number TEXT, \n\tsection_text TEXT, \n\tsource_url 
TEXT\n)\n\n/*\nColumn Comments: {'section_text': 'The text of the statute section. Chapter and article text can be 
constructed by aggregating the section texts.'}\n*/\n\n/*\n3 rows from nc_general_statutes 
table:\nchapter_number\tchapter_title\tarticle_number\tarticle\tsection_number\tsection_text\tsource_url\n1\tCivil 
Procedure\t1\tDefinitions\t1-1\t§ 1-1. \xa0Remedies.\nRemedies in the courts of justice are divided into 
-\n(1)\tActions.\n(2)\tSpecial 
proc\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-2\t§ 1-2. \xa0Actions.\nAn action is an ordinary proceeding in a court of justice, by 
which a party prosecu\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n1\tCivil 
Procedure\t1\tDefinitions\t1-3\t§ 1-3. \xa0Special proceedings.\nEvery other remedy is a special proceeding. 
(C.C.P., s. 3; Code, s. 
127\thttps://www.ncleg.gov/EnactedLegislation/Statutes/HTML/ByChapter/Chapter_1.html\n*/\n\nIf the user asks about 
a 'statute', assume they mean 'section'.\n\nEach row in the nc_general_statutes table is a statute section.\nThe 
values in the chapter_number and chapter_title columns will be duplicated for all statute sections belonging to the
same chapter. Use SELECT DISTINCT where necessary to get distinct values from these columns.\nThe values in the 
article_number and article columns will be duplicated for all statute sections belonging to the same article. Use 
SELECT DISTINCT where necessary to get distinct values from these columns."))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will search for the text of statute 15A-145 
in the nc_general_statutes table.'), ToolCallPart(tool_name='final_result_validate_sql_query', args={'sql_query': 
"SELECT section_text FROM nc_general_statutes WHERE section_number = '15A-145';"}, 
tool_call_id='tooluse_Lc4mHPgfRaGbuO0-ulYsLg')], usage=RequestUsage(input_tokens=649, output_tokens=66), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 31, 2, 315783, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='21e758be-ff33-4222-9f53-dbb1402ec4e6', finish_reason='tool_call'))

End(
    data=FinalResult(output="SELECT section_text FROM nc_general_statutes WHERE section_number = '15A-145';", 
tool_name='final_result_validate_sql_query', tool_call_id='tooluse_Lc4mHPgfRaGbuO0-ulYsLg')
)

...

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I will run the SQL query provided by the user 
and write an answer in simple language.'), ToolCallPart(tool_name='run_sql_query', args={'sql_query': "SELECT 
section_text FROM nc_general_statutes WHERE section_number = '15A-145';"}, 
tool_call_id='tooluse_I9pFt_cMQq2QOYNFF2hYoQ')], usage=RequestUsage(input_tokens=197, output_tokens=54), 
model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 31, 5, 766618, 
tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'tool_use'}, 
provider_response_id='79c58915-4f9e-4513-ac12-b61bfb904114', finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='run_sql_query', content=[("§ 15A-145. 
\xa0Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; 
expunction of certain other misdemeanors.\n(a)\tWhenever any person who has not previously been convicted of any 
felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State 
or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the 
offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a 
misdemeanor possession of alcohol pursuant to G.S.\xa018B-302(b)(1), and the offense was committed before the 
person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for 
expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years 
after the date of the conviction, or (ii) the completion of any period of probation, whichever occurs later, and 
the petition shall contain, but not be limited to, the following:\n(1)\tAn affidavit by the petitioner that he has 
been of good behavior for the two-year period since the date of conviction of the misdemeanor in question and has 
not been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United 
States or the laws of this State or any other state.\n(2)\tVerified affidavits of two persons who are not related 
to the petitioner or to each other by blood or marriage, that they know the character and reputation of the 
petitioner in the community in which he lives and that his character and reputation are good.\n(3)\tA statement 
that the petition is a motion in the cause in the case wherein the petitioner was convicted.\n(4)\tRepealed by 
Session Laws 2010-174, s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or 
after that date.\n(4a)\tAn application on a form approved by the Administrative Office of the Courts requesting and
authorizing a name-based State and national criminal record check by the State Bureau of Investigation using any 
information required by the Administrative Office of the Courts to identify the individual and a search of the 
confidential record of expunctions maintained by the Administrative Office of the Courts. The application shall be 
filed with the clerk of superior court. The clerk of superior court shall forward the application to the State 
Bureau of Investigation and to the Administrative Office of the Courts, which shall conduct the searches and report
their findings to the court.\n(5)\tAn affidavit by the petitioner that no restitution orders or civil judgments 
representing amounts ordered for restitution entered against him are outstanding.\nThe petition shall be served 
upon the district attorney of the court wherein the case was tried resulting in conviction. The district attorney 
shall have 10 days thereafter in which to file any objection thereto and shall be duly notified as to the date of 
the hearing of the petition.\nThe judge to whom the petition is presented is authorized to call upon a probation 
officer for any additional investigation or verification of the petitioner's conduct during the two-year period 
that he deems desirable.\n(a1)\tNothing in this section shall be interpreted to allow the expunction of any offense
involving impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration pursuant to 
Article 27A of Chapter 14 of the General Statutes, whether or not the person is currently required to 
register.\n(b)\tIf the court, after hearing, finds that the petitioner had remained of good behavior and been free 
of conviction of any felony or misdemeanor, other than a traffic violation, for two years from the date of 
conviction of the misdemeanor in question,

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="Statute 15A-145 is about people who have been 
convicted of a crime and want to get their criminal record deleted. If someone is under 18 and has been convicted 
of a crime for the first time, they can ask a court to remove the crime from their record. They have to wait for 
two years after the conviction and show that they have been behaving well during that time. The court will then 
decide if the crime should be removed from the person's record.")], usage=RequestUsage(input_tokens=2197, 
output_tokens=100), model_name='cohere.command-r-plus-v1:0', timestamp=datetime.datetime(2025, 11, 7, 13, 31, 14, 
998570, tzinfo=datetime.timezone.utc), provider_name='bedrock', provider_details={'finish_reason': 'end_turn'}, 
provider_response_id='6e3ddb24-45eb-4da7-bca4-356a94cb6cdd', finish_reason='stop'))

End(
    data=FinalResult(output="Statute 15A-145 is about people who have been convicted of a crime and want to get 
their criminal record deleted. If someone is under 18 and has been convicted of a crime for the first time, they 
can ask a court to remove the crime from their record. They have to wait for two years after the conviction and 
show that they have been behaving well during that time. The court will then decide if the crime should be removed 
from the person's record.")
)

Statute 15A-145 is about people who have been convicted of a crime and want to get their criminal record deleted. If someone is under 18 and has been convicted of a crime for the first time, they can ask a court to remove the crime from their record. They have to wait for two years after the conviction and show that they have been behaving well during that time. The court will then decide if the crime should be removed from the person's record.
